# COVID 19 automation serverless data pipeline
<a href="https://datastudio.google.com/s/qSPC32qTzPs"> Click here to navigate to the dashboard </a> <br>
| Table of Content |
| --- |
| I/ Purpose of the project |
| II/ Stages, tech stacks & status for project |
| III/ Instant set up |
| IV/ Question & My Solution |
_____
## I/ Purpose of the project
In the context of the COVID-19 pandemic, my project - a website/ dashboard that would be easy for everybody to follow the latest global figures as well as get more useful information such as:
* The social (data) analyst wants to show their insider about the epidemic status
* The journalist wants to get the information about future of the pandemic by reading the prediction part as a reference.
* Everyone who lives in affected countries could recognize then protect themself from the danger zone as well as release the pressure of mental health (worry, fear, etc) after seeing the downtrend of the epidemic.

## II/ Stages, tech stacks & status for project

| Stage | Tech stacks | Status |
| ----------- | ----------- | ----------- |
| First | Python, Google Bigquery API, Bigquery (Google Cloud Platform), SQL, Data Studio, Flourish, Facebook Prophet, Tableau Desktop version(optional) | ✓ Done|
| Second | Add more useful information to the dashboard | ✓ Done |
| Third | Predict cases | ✓ Done |

* Note:
1. The fifth stage, I need time to research.
2. The first stage, We can use Tableau desktop version to connect with Bigquery for automation the racing chart, but Flourish is free and it is pretty awesome!

## III/ Instant set up
### Environment Installation
Anaconda or Pycharm. Creating environment with Pycharm is easy to do. If with Anaconda:
```
$ pwd
$ cd ...                              [to the project dir ]
$ EXPORT PATH = "...."                [path to your anaconda]
$ conda create -n <envs> python=3.7   [python version should be over 3.0 to use the cutting-edge API/ modules]
$ source activate <envs>
$ (<envs>) python --version           [check version, it should be 3.7.4]
```

### Used modules
`$ pip install numpy pandas matplotlib seaborn io requests datetime `
### Google Bigquery API & Facebook Prophet
`$ pip install google-cloud-bigquery fbprophet` <br>
if it does not work for Google Bigquery, use this command instead:
> `$ conda install pandas-gbq --channel conda-forge`
### Datasource & Approach
Johns Hopkins University: <a href="https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series">JHU CSSE COVID-19 Dataset</a>
> The Python script includes functions to retrieve the latest data, transform & add new features for ad-hoc analysis as well as the suitable form for visualizing.

## IV/ Question & My Solution
##### Automation?
YES, the only manual-thing is that:
* need to re-run the python code and click the 'refresh' button in Data Studio to update the latest data from source and transformed data to new ad-hoc features -> could be totally automated if implement the fifth stage!
* update data for the racing chart on Flourish -> could be automated if use Tableau Desktop version to connect with Bigquery and set the scheduled query.
##### Why Flourish instead of Tableau?
Flourish is free and it is pretty awesome, Tableau costs expensive $70/month with the personal package - Tableau Creator.
##### Why Google Bigquery as a Data warehouse?
I can use IBM-Db2 with IBM Watson Studio, but the `ibm-db` module has conflicted with my local machine environment - cannot execute the module. Besides that, the most important thing is:
Google Bigquery is a serverless data warehouse (SaaS) that is a highly scalable, cost-effective, and real-time analysis with great performance. You not only can run a query to analyze terabytes-petabytes of data within seconds but also use it effectively with friendly user experience on Google Cloud Platform.
##### I need to see your source code as a referrence!
Updated


In [ ]:
import numpy as np
import pandas as pd
import io
import requests
import matplotlib.pyplot as plt
import datetime
from pandas.io import gbq
import os
from fbprophet import Prophet

In [ ]:
# set the starting time to calculate the eslapsing time
start_time = datetime.datetime.now()

## Create the DAILY DATA and LOAD it to the Datawarehouse

In [ ]:
# Auto update data source from csv to dataframe - daily data in global

# previous_Date in string with format YYYY-MM-DD HH:MM:SS.ffffff
def process_daily_data(n):
    previous_Date = datetime.datetime.today() - datetime.timedelta(days=n)
    previous_Date = str(previous_Date)
    #print('The latest update of daily data is the current date: {} \nThe date which is named in the url, has the previous date: {}\n So I create the url with the previous date form'.format(datetime.datetime.today(), previous_Date))

    get_year = previous_Date[:4]
    get_month = previous_Date[5:7]
    get_day = previous_Date[8:10]

    # change into this format: MM-DD-YYYY
    date_right_format = get_month + '-' + get_day + '-' + get_year + '.csv'
    url_daily_source = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
    url_daily_source_full = url_daily_source + date_right_format
    daily = requests.get(url_daily_source_full).content
    daily_data = pd.read_csv(io.StringIO(daily.decode('utf-8')))
    # even the new day has come, the date in the url is stil far away at least 1 or 2 days from the current day
    # But in case the updated data from JHU comes so late - over 2 days!
    # So I use the recursion here
    if str(daily_data) == 'Empty DataFrame\nColumns: [404: Not Found]\nIndex: []' :
        return process_daily_data(n+1)
    return daily_data, url_daily_source_full, get_month + '/' + get_day + '/' + get_year[2:]

daily_data, available_latest_url, date_format_in_dataset = process_daily_data(1)
list_daily_data_source = 'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports'
print('Available latest url: {} \n which is in the list of this source: {}\n'.format(available_latest_url, list_daily_data_source))
print('date format in dataset - use for the query {}'.format(date_format_in_dataset))
daily_data.head(2)

Available latest url: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/11-09-2020.csv 
 which is in the list of this source: https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

date format in dataset - use for the query 11/09/20


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-10 05:40:07,33.93911,67.709953,42297,1574,34721,6002.0,Afghanistan,108.653487,3.721304
1,NaN,NaN,NaN,Albania,2020-11-10 05:40:07,41.15330,20.168300,24731,571,12203,11957.0,Albania,859.371742,2.308843


In [ ]:
# need to change the name of the dataframe
daily_data = daily_data.rename(columns = {'Case-Fatality_Ratio':"Case_Fatality_Ratio"})

In [ ]:
# depend on the location of the file
os.getcwd()
#should be here

'/content'

In [ ]:
daily_data.to_gbq(destination_table='covid19data.covid19dailydata', project_id='covid19ds', if_exists='replace')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=Vdk4kKb2WckZajIzt4VtWLYjFJOOMv&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g6p3L4yO1GTnyUpVXGiZm8aVjvRs5_mhsOHDF2ZSyGOPNx-anX6L5s


1it [00:05,  5.02s/it]


In [ ]:
query = '''SELECT * FROM `covid19ds.covid19data.covid19dailydata`
WHERE Country_Region = 'Vietnam'
'''

In [ ]:
project_id='covid19ds'

In [ ]:
daily_data_query1 = gbq.read_gbq(query, project_id)

In [ ]:
daily_data_query1['Country_Region'].iloc[0]

'Vietnam'

In [ ]:
# import dashboard or flourish to jupyter notebook

# _____________
#from IPython.display import IFrame
# iframe data studio to python
# IFrame('https://datastudio.google.com/embed/reporting/4a60dfc8-b3d8-4148-838c-4ef6b5fc28c8/page/wSoOB', width='100%', height='600')

# _____________

# 1 & 2 for IFrame - python, 3 for html
#1/ flourish_url = 'https://public.flourish.studio/visualisation/2388175/'
#2/ IFrame(flourish_url, width='100%', height='600')
#3/ code embed in html: <div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/2388175" data-url="https://flo.uri.sh/visualisation/2388175/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>

## Create the TIME SERIES dataset and LOAD it to the Datawarehouse

In [ ]:
# fix url
url_confirmed_global = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url_death_global = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
url_recovered_global = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
confirmed = requests.get(url_confirmed_global).content
death = requests.get(url_death_global).content
recovered =requests.get(url_recovered_global).content
data_crfm = pd.read_csv(io.StringIO(confirmed.decode('utf-8')))
data_dead = pd.read_csv(io.StringIO(death.decode('utf-8')))
data_reco = pd.read_csv(io.StringIO(recovered.decode('utf-8')))
bar_chart_race_data = data_crfm


In [ ]:
data_crfm.to_csv('test2.csv')

In [ ]:
#What I am doing here is aloting the previous value to next date if the cumulative count is less on next date!
def modifier(x):
    return(x[0] if x[0]>x[1] else x[1])

def data_correctr(data):
    total_cols = data.shape[1]
    cols = data.columns
    for i in range(5,total_cols):
        data[cols[i]] = data[[cols[i-1], cols[i]]].apply(modifier, 1)
    return data

# df_t.rename(columns=df_t.iloc[0]).drop(df_t.index[0])

In [ ]:
#getting corrected data set!
data_crfm_c = data_correctr(data_crfm)
data_dead_c = data_correctr(data_dead)
data_reco_c = data_correctr(data_reco)


In [ ]:
total_cols = data_crfm_c.shape[1]

data_crfm_d = data_crfm_c.copy()
data_dead_d = data_dead_c.copy()
data_reco_d = data_reco_c.copy()

# this is done to calculate the percentage for every day (initalising day 1 to zero)
data_crfm_p = data_crfm_c.copy()
data_crfm_p.iloc[:,4] = 0
data_dead_p = data_dead_c.copy()
data_dead_p.iloc[:,4] = 0
data_reco_p = data_reco_c.copy()
data_reco_p.iloc[:,4] = 0


for i in range(5,total_cols):

    #converting cumulative to daily count
    data_crfm_d.iloc[:, i] = data_crfm_d.iloc[:, i] - data_crfm_c.iloc[:, i-1]
    data_dead_d.iloc[:, i] = data_dead_d.iloc[:, i] - data_dead_c.iloc[:, i-1]
    data_reco_d.iloc[:, i] = data_reco_d.iloc[:, i] - data_reco_c.iloc[:, i-1]

    #percentage change: I will store the previous day cumulative and apply percentage change later
    data_crfm_p.iloc[:, i] = data_crfm_c.iloc[:, i-1]
    data_dead_p.iloc[:, i] = data_dead_c.iloc[:, i-1]
    data_reco_p.iloc[:, i] = data_reco_c.iloc[:, i-1]

# Here I am storing previous day daily count I will need this to calculate percentage change metric: the 6 small box in the dashboard
data_crfm_dp = data_crfm_d.copy()
data_crfm_dp.iloc[:,4] = 0
data_dead_dp = data_dead_d.copy()
data_dead_dp.iloc[:,4] = 0
data_reco_dp = data_reco_d.copy()
data_reco_dp.iloc[:,4] = 0

for i in range(5,total_cols):
    #percentage change: I will store the previous day daily and apply percentage change later
    data_crfm_dp.iloc[:, i] = data_crfm_d.iloc[:, i-1]
    data_dead_dp.iloc[:, i] = data_dead_d.iloc[:, i-1]
    data_reco_dp.iloc[:, i] = data_reco_d.iloc[:, i-1]

In [ ]:
data_crfm_d.head(1)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,17,5,7,44,81,64,62,68,77,10,96,71,58,66,32,47,68,59,87,70,153,116,61,81,65,104,95,113,123,66,91,76,132,95,86,121,40,58,59,205


In [ ]:
# Here comes the melt funtion of pandas. One line and your columns turns into rows!
df_crfm = pd.melt(data_crfm_d, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"Daily_Confirmed"})

In [ ]:
df_crfm.head()

,Province/State,Country/Region,Lat,Long,Time,Daily_Confirmed
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [ ]:
# continue with other columns and
# must rename for suitable with form of Google Big Query
df_dead = pd.melt(data_dead_d, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"Daily_Death"})
df_reco = pd.melt(data_reco_d, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"Daily_Recovered"})

df_crfm_c = pd.melt(data_crfm_c, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"Cum_Confirmed"})
df_dead_c = pd.melt(data_dead_c, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"Cum_Death"})
df_reco_c = pd.melt(data_reco_c, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"Cum_Recovered"})

df_crfm_p = pd.melt(data_crfm_p, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"PCum_Confirmed"})
df_dead_p = pd.melt(data_dead_p, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"PCum_Death"})
df_reco_p = pd.melt(data_reco_p, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"PCum_Recovered"})

df_crfm_dp = pd.melt(data_crfm_dp, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"dPCum_Confirmed"})
df_dead_dp = pd.melt(data_dead_dp, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"dPCum_Death"})
df_reco_dp = pd.melt(data_reco_dp, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'Time').rename(columns = {'value':"dPCum_Recovered"})

In [ ]:
print(df_crfm.shape, df_dead.shape, df_reco.shape, df_crfm_c.shape, df_dead_c.shape, df_reco_c.shape, df_crfm_p.shape, df_dead_p.shape, df_reco_p.shape)

(78524, 6) (78524, 6) (74715, 6) (78524, 6) (78524, 6) (74715, 6) (78524, 6) (78524, 6) (74715, 6)


In [ ]:
#Collecting the metric into 1 data frame
df = df_crfm.merge(df_dead[['Country/Region','Lat','Long', 'Time', 'Daily_Death']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])
df = df.merge(df_reco[['Country/Region','Lat','Long', 'Time', 'Daily_Recovered']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])

df = df.merge(df_crfm_c[['Country/Region','Lat','Long', 'Time', 'Cum_Confirmed']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])
df = df.merge(df_dead_c[['Country/Region','Lat','Long', 'Time', 'Cum_Death']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])
df = df.merge(df_reco_c[['Country/Region','Lat','Long', 'Time', 'Cum_Recovered']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])

df = df.merge(df_crfm_p[['Country/Region','Lat','Long', 'Time', 'PCum_Confirmed']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])
df = df.merge(df_dead_p[['Country/Region','Lat','Long', 'Time', 'PCum_Death']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])
df = df.merge(df_reco_p[['Country/Region','Lat','Long', 'Time', 'PCum_Recovered']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])

df = df.merge(df_crfm_dp[['Country/Region','Lat','Long', 'Time', 'dPCum_Confirmed']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])
df = df.merge(df_dead_dp[['Country/Region','Lat','Long', 'Time', 'dPCum_Death']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])
df = df.merge(df_reco_dp[['Country/Region','Lat','Long', 'Time', 'dPCum_Recovered']], how = 'left', on = ['Country/Region','Lat', 'Long', 'Time'])

In [ ]:
df.head()

,Province/State,Country/Region,Lat,Long,Time,Daily_Confirmed,Daily_Death,Daily_Recovered,Cum_Confirmed,Cum_Death,Cum_Recovered,PCum_Confirmed,PCum_Death,PCum_Recovered,dPCum_Confirmed,dPCum_Death,dPCum_Recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0
1,NaN,Albania,41.15330,20.168300,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0


In [ ]:
#last datatype corrections before feeding to data studio
df['Lat_Long'] = df['Lat'].astype(str)+","+df['Long'].astype(str)
df.head()

,Province/State,Country/Region,Lat,Long,Time,Daily_Confirmed,Daily_Death,Daily_Recovered,Cum_Confirmed,Cum_Death,Cum_Recovered,PCum_Confirmed,PCum_Death,PCum_Recovered,dPCum_Confirmed,dPCum_Death,dPCum_Recovered,Lat_Long
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,"33.93911,67.709953"
1,NaN,Albania,41.15330,20.168300,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,"41.1533,20.1683"
2,NaN,Algeria,28.03390,1.659600,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,"28.0339,1.6596"
3,NaN,Andorra,42.50630,1.521800,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,"42.5063,1.5218"
4,NaN,Angola,-11.20270,17.873900,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,"-11.2027,17.8739"


In [ ]:
df['Death_Rate'] = ((df['Cum_Death'] *100) /df['Cum_Confirmed']).replace([np.inf, -np.inf], np.nan).fillna(0)
df['Recovered_Rate'] = ((df['Cum_Recovered'] *100) /df['Cum_Confirmed']).replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:

tsd = df.rename(columns = {'Country/Region':"Country_Region",
                           'Province/State':'Province_State',
                           'Cum_Confirmed':'acc_Confirmed',
                           'Cum_Death':'acc_Death',
                           'Cum_Recovered':'acc_Recovered',
                           'PCum_Confirmed':'pacc_confirmed',
                           'PCum_Death':'pacc_Death',
                           'PCum_Recovered':'pacc_Recovered',
                           'dPCum_Confirmed':'dpacc_Confirmed',
                           'dPCum_Death':'dpacc_Death',
                           'dPCum_Recovered':'dpacc_Recovered',

                          })
tsd.head(1)

,Province_State,Country_Region,Lat,Long,Time,Daily_Confirmed,Daily_Death,Daily_Recovered,acc_Confirmed,acc_Death,acc_Recovered,pacc_confirmed,pacc_Death,pacc_Recovered,dpacc_Confirmed,dpacc_Death,dpacc_Recovered,Lat_Long,Death_Rate,Recovered_Rate
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,"33.93911,67.709953",0.0,0.0


In [ ]:
tsd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152946 entries, 0 to 152945
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Province_State   98155 non-null   object 
 1   Country_Region   152946 non-null  object 
 2   Lat              152946 non-null  float64
 3   Long             152946 non-null  float64
 4   Time             152946 non-null  object 
 5   Daily_Confirmed  152946 non-null  int64  
 6   Daily_Death      152946 non-null  int64  
 7   Daily_Recovered  73250 non-null   float64
 8   acc_Confirmed    152946 non-null  int64  
 9   acc_Death        152946 non-null  int64  
 10  acc_Recovered    73250 non-null   float64
 11  pacc_confirmed   152946 non-null  int64  
 12  pacc_Death       152946 non-null  int64  
 13  pacc_Recovered   73250 non-null   float64
 14  dpacc_Confirmed  152946 non-null  int64  
 15  dpacc_Death      152946 non-null  int64  
 16  dpacc_Recovered  73250 non-null   floa

In [ ]:
50850*2

101700

In [ ]:

tsd[tsd['Country_Region'] == 'Vietnam'].tail(15).isnull()

,Province_State,Country_Region,Lat,Long,Time,Daily_Confirmed,Daily_Death,Daily_Recovered,acc_Confirmed,acc_Death,acc_Recovered,pacc_confirmed,pacc_Death,pacc_Recovered,dpacc_Confirmed,dpacc_Death,dpacc_Recovered,Lat_Long,Death_Rate,Recovered_Rate
145632,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
146154,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
146676,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
147198,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
147720,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
148242,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
148764,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
149286,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
149808,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
150330,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
tsd.to_gbq(destination_table='covid19data.covid19tsd', project_id='covid19ds', if_exists='replace')

1it [00:23, 23.27s/it]


In [ ]:
os.getcwd()

'/content'

## Create a new Table for the LATEST recovered & death rate

## Prepare data for the race bar chart

In [ ]:
query = '''SELECT Country_Region, SUM(Confirmed) as Total_Confirmed
FROM `covid19ds.covid19data.covid19dailydata`
GROUP BY Country_Region
ORDER BY Total_Confirmed DESC
LIMIT 10'''

In [ ]:
# test another case: recovered >= 40k and <= 100000
query = '''SELECT Country_Region, SUM(Recovered) as Total_Recovered
FROM `covid19ds.covid19data.covid19dailydata`
WHERE Confirmed >= 40000 and Confirmed <=100000 and Country_Region != 'China'
GROUP BY Country_Region
ORDER BY Total_Recovered DESC
LIMIT 10'''

In [ ]:
data_q = gbq.read_gbq(query, project_id)

In [ ]:
data_q

,Country_Region,Total_Recovered
0,Brazil,525324
1,Colombia,232703
2,India,195376
3,Russia,149541
4,Italy,103968
5,Mexico,94659
6,Germany,91686
7,Venezuela,90054
8,Bahrain,81225
9,Canada,75877


In [ ]:
# list the countries in data_q
data_q_list = list(data_q['Country_Region'])
# check
data_q_list[len(data_q_list)-1]

'Canada'

In [ ]:
# get the value of the TOP 10 recovered
data_q_value = list(data_q['Total_Recovered'])
rank_10_cf = data_q_value[len(data_q_value)-1]
print('Country which is at 10th rank: {} with {} Recovered cases'.format(data_q_list[len(data_q_list)-1], rank_10_cf))
# change bar_chart_race_data above
bar_chart_race_data = data_reco
rank_1_10_cf = data_q_value[0]
rank_1_10_cf

Country which is at 10th rank: Canada with 75877 Recovered cases


525324

In [ ]:
# get the value of the TOP 10 confirmed
#data_q_value = list(data_q['Total_Confirmed'])
#rank_10_cf = data_q_value[len(data_q_value)-1]
#print('Country which is at 10th rank: {} with {} confirmed cases'.format(data_q_list[len(data_q_list)-1], rank_10_cf))


In [ ]:
# create the right name of the final dimensions of the dataset - it's the latest updated day also!
form_d = int(date_format_in_dataset[0:2])
form_m = int(date_format_in_dataset[3:5])
condition_top10 = str(form_d) + '/' + str(form_m) + '/' + date_format_in_dataset[6:]
condition_top10
bar_chart_race_data['Country/Region']

0             Afghanistan
1                 Albania
2                 Algeria
3                 Andorra
4                  Angola
              ...        
250    West Bank and Gaza
251        Western Sahara
252                 Yemen
253                Zambia
254              Zimbabwe
Name: Country/Region, Length: 255, dtype: object

In [ ]:
top_10_tsd = bar_chart_race_data[(bar_chart_race_data['Country/Region'] == data_q_list[0]) |
                                 (bar_chart_race_data['Country/Region'] == data_q_list[1]) |
                                 (bar_chart_race_data['Country/Region'] == data_q_list[2]) |
                                 (bar_chart_race_data['Country/Region'] == data_q_list[3]) |
                                 (bar_chart_race_data['Country/Region'] == data_q_list[4]) |
                                 (bar_chart_race_data['Country/Region'] == data_q_list[5]) |
                                 (bar_chart_race_data['Country/Region'] == data_q_list[6]) |
                                 (bar_chart_race_data['Country/Region'] == data_q_list[7]) |
                                 (bar_chart_race_data['Country/Region'] == data_q_list[8]) |
                                 (bar_chart_race_data['Country/Region'] == data_q_list[9])]
top_10_tsd

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20
19,NaN,Bahrain,26.027500,50.550000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,65550,66184,66813,67475,67933,68606,69411,69411,70406,70808,71249,71687,72164,72561,73013,73421,73841,74320,74683,75089,75424,75840,76143,76474,76776,77132,77421,77697,78102,78365,78719,78997,79318,79680,79929,80174,80526,80763,81000,81225
30,NaN,Brazil,-14.235000,-51.925300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4299659,4299659,4361108,4375354,4397882,4430340,4457172,4488092,4502854,4502854,4526393,4526393,4526393,4526393,4526393,4526393,4526393,4526393,4526393,4526393,4526393,4526393,4526393,4526393,4526393,4526393,4914733,4947777,4962574,4987435,5005161,5022338,5039438,5060697,5078162,5078162,5118156,5138641,5147172,5163226
39,NaN,Canada,56.130400,-106.346800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,...,138548,139817,140711,142502,144653,146429,147814,149692,152046,153857,155901,156940,160223,162064,164154,166313,166444,169751,172406,174514,174514,174514,180626,181974,184194,187100,189350,191723,194105,196935,198952,200768,203128,206691,208815,210429,214441,216338,218279,221758
76,NaN,Colombia,4.570900,-74.297300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,751691,753953,757801,761674,766300,770812,773973,773973,780547,783131,789787,798396,806703,816667,826831,837001,847467,858294,867961,876731,884895,893810,901652,907379,907379,924044,932882,941874,950348,959433,969230,977804,985796,993877,1002202,1011166,1020263,1029082,1038082,1047017
117,NaN,Germany,51.165691,10.451526,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,12,12,12,14,14,14,14,14,15,...,258220,259722,260916,262002,265624,267747,269722,271960,273718,274934,276983,278391,280533,283656,286137,289027,291129,293447,296672,299743,304173,308779,312347,316124,320004,325052,327697,336764,340436,351921,356410,363545,372499,373391,384012,394616,405809,413484,421151,443621
130,NaN,India,20.593684,78.962880,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,3,3,3,3,3,3,3,3,3,...,5352078,5427706,5509966,5586703,5662490,5744693,5827704,5906069,5988822,6077976,6149535,6227295,6301927,6383441,6453779,6524595,6597209,6663608,6733328,6795103,6874518,6948497,7016046,7078123,7137228,7201070,7259509,7315989,7373375,7432829,7491513,7544798,7603121,7656478,7711809,7765966,7819886,7868968,7917373,7959406
136,NaN,Italy,41.871940,12.567380,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,3,...,228844,229970,231217,231914,232681,234099,235303,236363,237549,238525,239709,240600,242028,244065,245964,247872,249127,251461,252959,255005,257374,259456,261808,264117,266203,268626,271988,275404,279282,283567,289426,292380,296017,302275,307378,312339,322925,328891,335074,345289
165,NaN,Mexico,23.634500,-102.552800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,629630,633806,637993,645123,670034,676923,679693,682604,687123,689377,694214,700826,703457,703489,707631,712250,715848,720973,727759,730594,733897,737437,740934,746939,752691,760259,763105,765128,770007,774567,779371,786516,795356,803086,804887,809063,812556,816659,824355,824355
196,NaN,Russia,61.524010,105.318756,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...,960729,966724,972249,975488,978610,9

In [ ]:
top_10_tsd.to_csv('test.csv')

In [ ]:
# use the tsd dataframe
tsd.head(2)
print("")
tsd[tsd['Country_Region'] == 'Vietnam']

,Province_State,Country_Region,Lat,Long,Time,Daily_Confirmed,Daily_Death,Daily_Recovered,acc_Confirmed,acc_Death,acc_Recovered,pacc_confirmed,pacc_Death,pacc_Recovered,dpacc_Confirmed,dpacc_Death,dpacc_Recovered,Lat_Long,Death_Rate,Recovered_Rate
516,NaN,Vietnam,14.058324,108.277199,1/22/20,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,"14.058323999999999,108.277199",0.000000,0.000000
1038,NaN,Vietnam,14.058324,108.277199,1/23/20,2,0,0.0,2,0,0.0,0,0,0.0,0,0,0.0,"14.058323999999999,108.277199",0.000000,0.000000
1560,NaN,Vietnam,14.058324,108.277199,1/24/20,0,0,0.0,2,0,0.0,2,0,0.0,2,0,0.0,"14.058323999999999,108.277199",0.000000,0.000000
2082,NaN,Vietnam,14.058324,108.277199,1/25/20,0,0,0.0,2,0,0.0,2,0,0.0,0,0,0.0,"14.058323999999999,108.277199",0.000000,0.000000
2604,NaN,Vietnam,14.058324,108.277199,1/26/20,0,0,0.0,2,0,0.0,2,0,0.0,0,0,0.0,"14.058323999999999,108.277199",0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150852,NaN,Vietnam,14.058324,108.277199,11/5/20,4,0,0.0,1207,35,1069.0,1203,35,1069.0,1,0,0.0,"14.058323999999999,108.277199",2.899751,88.566694
151374,NaN,Vietnam,14.058324,108.277199,11/6/20,5,0,1.0,1212,35,1070.0,1207,35,1069.0,4,0,0.0,"14.058323999999999,108.277199",2.887789,88.283828
151896,NaN,Vietnam,14.058324,108.277199,11/7/20,1,0,0.0,1213,35,1070.0,1212,35,1070.0,5,0,1.0,"14.058323999999999,108.277199",2.885408,88.211047
152418,NaN,Vietnam,14.058324,108.277199,11/8/20,0,0,0.0,1213,35,1070.0,1213,35,1070.0,1,0,0.0,"14.058323999999999,108.277199",2.885408,88.211047


## New features - [ad hoc analysis] Predict future with any metrics

In [ ]:
# create model and make prediction:
def predict_viz(range_day):
    # interval 0.63 for Vietnam, another: 0.91
    m = Prophet(interval_width=0.63,daily_seasonality=True)
    m.fit(df_fbprophet)
    future = m.make_future_dataframe(periods = range_day)
    forecast = m.predict(future)

    fig, ax = plt.subplots(facecolor='#F3F3F3')
    #fig = plt.figure()
    #fig.patch.set_facecolor('#F3F3F3')

    fig_predict_1 = m.plot(forecast, xlabel = 'Date - predict the next {} days'.format(range_day), ylabel = chosen_metrics)
    fig_predict_2 = m.plot_components(forecast)

    fig_predict_1.savefig('result1.jpg', facecolor=fig.get_facecolor(), transparent=True)
    fig_predict_2.savefig('result2.jpg', facecolor=fig.get_facecolor(), transparent=True)
    return forecast

# function to create a dataset with countries and metrics you want to predicts!
# And transform to the right form for using fbProphet
def create_df_fbprophet(ct, metrics):
    df_fbprophet = tsd[tsd['Country_Region'] == ct]
    df_fbprophet = df_fbprophet[['Time', metrics]].rename(columns = {'Time':'ds', metrics:'y'})
    return df_fbprophet


In [ ]:
# CHOOSE CT      0           1    2
list_ct = ['Vietnam', 'Finland', 'US']
# CHOOSE METRICS         0                1               2                 3
list_metrics = ['acc_Confirmed', 'Daily_Confirmed', 'Daily_Recovered', 'acc_Recovered']
# choose fix val - normaly VN 0 2, US 2 1
chosen_ct = list_ct[0]
chosen_metrics = list_metrics[2]

# use func
df_fbprophet = create_df_fbprophet(chosen_ct, chosen_metrics)
df_fbprophet.tail(20)
# df_fbprophet = df_fbprophet[int(len(df_fbprophet)/3):]
df_fbprophet

,ds,y
516,1/22/20,0.0
1038,1/23/20,0.0
1560,1/24/20,0.0
2082,1/25/20,0.0
2604,1/26/20,0.0
...,...,...
150852,11/5/20,0.0
151374,11/6/20,1.0
151896,11/7/20,0.0
152418,11/8/20,0.0


In [ ]:
import numpy as np
df_fbprophet.isnull()

,ds,y
516,False,False
1038,False,False
1560,False,False
2082,False,False
2604,False,False
...,...,...
150852,False,False
151374,False,False
151896,False,False
152418,False,False


In [ ]:
# show upper and lower limit of y value, choose the range of day wants to predict
# vietnam 14, US 45
range_day = 45
forecast = predict_viz(range_day)
# take the previous day
previous_day = 13
total_day = range_day + previous_day
forecast_df_tobq = forecast.tail(total_day)
forecast_df_tobq.to_gbq(destination_table='covid19data.testPredict', project_id='covid19ds', if_exists='replace')

In [ ]:
time_elapsed = datetime.datetime.now() - start_time
print('Time eslapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time eslapsed (hh:mm:ss.ms) 0:01:44.308702
